In [6]:
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_squared_error
import networkx as nx
import math
import sys
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('./OTCNet.csv')

N = 5

X =  df
#y =  df['rating']
X = X.values
#y = y.values

lpo = LeavePOut(N)
lpo.get_n_splits(X)

475837529743534588728

In [8]:
def initialize_scores_bd(G):
    bias = {}
    deserve = {}
    
    nodes = G.nodes()
    #print(nodes)
    for node in nodes:
        bias[node] = -1
        try:
            deserve[node] = G.in_degree(node, weight='weight')*1.0/G.in_degree(node)
        except:
            deserve[node] = 0
    return bias, deserve

In [9]:
def compute_bias_deserve(G):
    bias, deserve = initialize_scores_bd(G)
    
    nodes = G.nodes()
    iter = 0
    while iter < 100:
        d = 0
        db = 0
        
        for node in nodes:
            inedges = G.in_edges(node, data='weight')
            #print(inedges)
            d = 0
            for edge in inedges:
                Xkj = max(0, bias[node]*edge[2])
                d += edge[2]*(1 - Xkj)
            try:
                dd += abs(d/len(inedges) - deserve[node])
                deserve[node] = d/len(inedges)
            except:
                pass

        for node in nodes:
            outedges = G.out_edges(node, data='weight')
            b = 0
            for edge in outedges: 
                b += (edge[2] - deserve[edge[1]])
            try:
                db += abs(b/(2*len(outedges)) - bias[node])
                bias[node] = b/(2*len(outedges))
            except:
                pass
        
        
        if db < math.pow(10, -6):
            break
        #print('Differences in bias score score = %.2f' % db)
        iter+=1
    
    return bias, deserve

In [10]:
# only for first 20 iterations
itr = 10
bias = {}
deserve = {}
predicted = pd.DataFrame(np.random.randn(5, 1), columns=['rat'])
test = pd.DataFrame(np.random.randn(5, 1), columns=['rat'])

goodness_test = {}
RMSE = 0
PCC = 0
for train_index, test_index in lpo.split(X):
    if test_index[4] < (itr+N):
        X_train, X_test = X[train_index], X[test_index]
        G = nx.DiGraph()
        test['rat'][0] = X_test[0][2]
        test['rat'][1] = X_test[1][2]
        test['rat'][2] = X_test[2][2]
        test['rat'][3] = X_test[3][2]
        test['rat'][4] = X_test[4][2]
        for index in range(0, 35587):
            G.add_edge(X_train[index][0], X_train[index][1], weight = float(X_train[index][2])) ## the weight should already be in the range of -1 to 1
        try:
            bias, deserve = compute_bias_deserve(G)
        except:
            pass
        try:
            predicted['rat'][0] = deserve[X_test[0][1]]
        except:
            predicted['rat'][0] = 0
            continue
        try:
            predicted['rat'][1] = deserve[X_test[1][1]]
        except:
            predicted['rat'][1] = 0
            continue
        try:
            predicted['rat'][2] = deserve[X_test[2][1]]
        except:
            predicted['rat'][2] = 0
            continue
        try:
            predicted['rat'][3] = deserve[X_test[3][1]]
        except:
            predicted['rat'][3] = 0
            continue
        try:
            predicted['rat'][4] = deserve[X_test[4][1]]
        except:
            predicted['rat'][4] = 0
            continue
    else:
        break
    RMSE += math.sqrt(mean_squared_error(test['rat'], predicted['rat']))
    PCC += np.corrcoef(test['rat'], predicted['rat'])
    print('iteration', test_index[4] - N)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [11]:
RMES_bd_leaveNout = RMSE/itr
RMES_bd_leaveNout

0.3986889987620701

In [12]:
PCC_bd_leaveNout = PCC/itr
PCC_bd_leaveNout

array([[ 1.        , -0.34813613],
       [-0.34813613,  1.        ]])